# Scraper to get the data for Chicago Zip Codes from the following site.

Zip Code List:
http://zipatlas.com/us/il/chicago/zip-code-comparison/population-density.htm

Pages with Zip Code Data:
http://zipatlas.com/us/il/chicago/zip-60649.htm

In [47]:
import pandas as pd
import numpy as np
import re

from bs4 import BeautifulSoup
from time import sleep
import random
import requests
import urllib
from html_table_parser import parser_functions as parse

In [10]:
#Scraper to get the table from the website with all the zip codes.
url = "http://zipatlas.com/us/il/chicago/zip-code-comparison/population-density.htm"
html = urllib.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')
table = soup.find('table', {'rules':'all'})
twodim_table = parse.make2d(table)
zip_df = pd.DataFrame(twodim_table)

In [15]:
#Adjusting the column headers on the dataframe.
zip_df.columns = zip_df.loc[0]
zip_df.drop(0, inplace=True)

In [111]:
print zip_df.shape
zip_df = zip_df.reset_index()
zip_df.drop(['index', '#'], axis=1, inplace=True)
zip_df.head()

(55, 7)


,Zip Code,Location,City,Population,People / Sq. Mile,National Rank
0,60626,"42.009340, -87.669902","Chicago, Illinois","59,251","34,520.21",#98
1,60660,"41.990561, -87.664706","Chicago, Illinois","47,726","34,193.05",#99
2,60640,"41.972347, -87.663982","Chicago, Illinois","74,030","32,103.29",#109
3,60657,"41.938964, -87.656811","Chicago, Illinois","66,789","31,204.02",#112
4,60610,"41.900034, -87.636967","Chicago, Illinois","47,513","30,110.68",#119


In [52]:
#Function to strip html which will be used in the details scraper.
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub(' ', data)

In [93]:
#Scraper to get the detailed data from each zip code page.
zip_detail = pd.DataFrame(columns=['Latitude', 'Longitude', 'Population', 'Areasqmile', 'Landsqmile', 'Watersqmile', 'Poppersqmile'])
for i in zip_df['Zip Code']:
    x = str(i)
    url = "http://zipatlas.com/us/il/chicago/zip-"+x+".htm"    
    html = urllib.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')
    table = soup.find('div', {'style':'padding:10px 10px 10px 25px;'})
    table = striphtml(str(table))
    table = table.split()
    #Scraped data stored in variables.
    Latitude = table[9]
    Longitude = table[11]
    Population = table[13]
    Areasqmile = table[15]
    Landsqmile = table[19]
    Watersqmile = table[23]
    Poppersqmile = table[28]
    zip_detail.loc[len(zip_detail)]=[Latitude, Longitude, Population, Areasqmile, Landsqmile, Watersqmile, Poppersqmile]
    print zip_detail.shape

(1, 7)
(2, 7)
(3, 7)
(4, 7)
(5, 7)
(6, 7)
(7, 7)
(8, 7)
(9, 7)
(10, 7)
(11, 7)
(12, 7)
(13, 7)
(14, 7)
(15, 7)
(16, 7)
(17, 7)
(18, 7)
(19, 7)
(20, 7)
(21, 7)
(22, 7)
(23, 7)
(24, 7)
(25, 7)
(26, 7)
(27, 7)
(28, 7)
(29, 7)
(30, 7)
(31, 7)
(32, 7)
(33, 7)
(34, 7)
(35, 7)
(36, 7)
(37, 7)
(38, 7)
(39, 7)
(40, 7)
(41, 7)
(42, 7)
(43, 7)
(44, 7)
(45, 7)
(46, 7)
(47, 7)
(48, 7)
(49, 7)
(50, 7)
(51, 7)
(52, 7)
(53, 7)
(54, 7)
(55, 7)


In [96]:
print zip_detail.shape
zip_detail.head(3)

(55, 7)


,Latitude,Longitude,Population,Areasqmile,Landsqmile,Watersqmile,Poppersqmile
0,42.0093402,-87.6699015,"59,251",1.72,1.72,0.00,"34,520.22"
1,41.9905605,-87.6647059,"47,726",1.40,1.40,0.00,"34,193.05"
2,41.9723469,-87.6639822,"74,030",2.31,2.31,0.00,"32,103.29"


In [112]:
#Concatenate the two dataframes together.
zip_scraper = pd.concat([zip_df, zip_detail], axis=1)

In [113]:
print zip_scraper.shape
zip_scraper.head()

(55, 13)


,Zip Code,Location,City,Population,People / Sq. Mile,National Rank,Latitude,Longitude,Population,Areasqmile,Landsqmile,Watersqmile,Poppersqmile
0,60626,"42.009340, -87.669902","Chicago, Illinois","59,251","34,520.21",#98,42.0093402,-87.6699015,"59,251",1.72,1.72,0.00,"34,520.22"
1,60660,"41.990561, -87.664706","Chicago, Illinois","47,726","34,193.05",#99,41.9905605,-87.6647059,"47,726",1.40,1.40,0.00,"34,193.05"
2,60640,"41.972347, -87.663982","Chicago, Illinois","74,030","32,103.29",#109,41.9723469,-87.6639822,"74,030",2.31,2.31,0.00,"32,103.29"
3,60657,"41.938964, -87.656811","Chicago, Illinois","66,789","31,204.02",#112,41.9389636,-87.6568112,"66,789",2.14,2.10,0.04,"31,834.77"
4,60610,"41.900034, -87.636967","Chicago, Illinois","47,513","30,110.68",#119,41.9000342,-87.6369674,"47,513",1.58,1.58,0.00,"30,110.68"


In [114]:
#Backup the scraped stuff in a csv.
zip_scraper.to_csv('assets/SCRAPED_ZIP_CODE_DETAILS.csv')